# Deepfake Face Swap: Gambar ke Gambar di Google Colab

Notebook ini memandu Anda untuk melakukan face swap dari satu gambar sumber ke gambar target menggunakan `insightface`.

## 1. Setup Environment

Instalasi library yang dibutuhkan.

In [ ]:
# CRUCIAL: Instalasi library utama
!pip install -q insightface==0.7.3 onnxruntime-gpu opencv-python-headless numpy tqdm

**PENTING:** Setelah menjalankan sel instalasi di atas, Anda **HARUS** me-restart runtime agar library yang baru diinstal dapat dikenali dengan benar. 
Caranya: Pilih menu `Runtime` -> `Restart runtime` (atau `Ctrl+M .`). Setelah itu, lanjutkan menjalankan sel berikutnya.

## 2. Import Libraries dan Inisialisasi Model Utama

Mengimpor library dan menginisialisasi model `FaceAnalysis` serta model `swapper`. Model swapper akan diunduh dari URL Hugging Face.

In [ ]:
import cv2
import numpy as np
import insightface # Library utama
from insightface.app import FaceAnalysis
import matplotlib.pyplot as plt 
import os
import subprocess # Untuk wget
import traceback # Untuk debugging
from google.colab import files # Untuk upload file Colab

# CRUCIAL: URL untuk model swapper dari Hugging Face (menggunakan link /resolve/ untuk unduhan langsung)
URL_MODEL_SWAPPER = "https://huggingface.co/ezioruan/inswapper_128.onnx/resolve/main/inswapper_128.onnx"

providers = [] # Akan diisi berdasarkan ketersediaan GPU
try:
    import onnxruntime
    print(f"ONNXRuntime versi: {onnxruntime.__version__}")
    print(f"ONNXRuntime provider tersedia: {onnxruntime.get_available_providers()}")
    if 'CUDAExecutionProvider' in onnxruntime.get_available_providers():
        providers = ['CUDAExecutionProvider', 'CPUExecutionProvider'] 
        print("CUDAExecutionProvider (GPU) akan digunakan.")
    else:
        providers = ['CPUExecutionProvider']
        print("CUDAExecutionProvider tidak ditemukan, menggunakan CPUExecutionProvider.")
except ImportError:
    print("ONNXRuntime tidak terinstal dengan benar.")
    providers = ['CPUExecutionProvider']

print(f"Provider ONNXRuntime yang akan digunakan: {providers}")
print(f"Insightface versi: {insightface.__version__}")

# CRUCIAL: Inisialisasi FaceAnalysis untuk deteksi wajah
print("\nInisialisasi FaceAnalysis (model buffalo_l)...")
app = FaceAnalysis(name='buffalo_l', root='~/.insightface', providers=providers)
app.prepare(ctx_id=0, det_size=(640, 640)) 
print("Model FaceAnalysis (buffalo_l) siap.")

# CRUCIAL: Inisialisasi model untuk face swapping
print("\nInisialisasi model swapper...")
swapper = None
try:
    models_dir = os.path.expanduser('~/.insightface/models')
    os.makedirs(models_dir, exist_ok=True)
    model_filename = 'inswapper_128.onnx'
    model_path = os.path.join(models_dir, model_filename)
    print(f"Path target untuk model swapper: {model_path}")

    if not URL_MODEL_SWAPPER or URL_MODEL_SWAPPER == "MASUKKAN_URL_LANGSUNG_KE_FILE_INSWAPPER_128_ONNX_DI_SINI":
        print("URL_MODEL_SWAPPER tidak valid. Tidak bisa mengunduh model.")
        if not os.path.exists(model_path):
             raise FileNotFoundError(f"Model swapper tidak ditemukan di {model_path} dan URL tidak disediakan.")
    else:
        print(f"URL untuk mengunduh model swapper: {URL_MODEL_SWAPPER}")
        if os.path.exists(model_path):
            file_size_mb = os.path.getsize(model_path) / (1024*1024)
            print(f"Model swapper lama ditemukan ({file_size_mb:.2f} MB). Menghapus sebelum mengunduh yang baru...")
            os.remove(model_path)
        
        print(f"Mengunduh model swapper dari URL ke '{model_path}'...")
        subprocess.run(['wget', '-nv', '--show-progress', '-O', model_path, URL_MODEL_SWAPPER], check=True)
        print("Model swapper berhasil diunduh.")
        
        if os.path.exists(model_path):
            file_size_mb = os.path.getsize(model_path) / (1024*1024)
            print(f"Ukuran file model yang diunduh: {file_size_mb:.2f} MB.")
            if file_size_mb < 40: 
                print("PERINGATAN: Ukuran file model yang diunduh sangat kecil!")
        else:
            raise FileNotFoundError("Model tidak ditemukan setelah proses unduh.")

    if os.path.exists(model_path):
        print(f"\nMencoba memuat model swapper dari: {model_path}")
        swapper = insightface.model_zoo.get_model(model_path, providers=providers)
        print("Model swapper berhasil dimuat.")
    else:
        print(f"File model swapper tidak ditemukan di '{model_path}'.")
        raise FileNotFoundError(f"Model swapper tidak ada di {model_path}.")

except Exception as e_model_init:
    print(f"\n--- ERROR SAAT INISIALISASI MODEL SWAPPER --- ")
    traceback.print_exc()
    swapper = None 

if app and swapper:
    print("\nModel FaceAnalysis (app) dan Swapper (swapper) berhasil diinisialisasi.")
else:
    print("\nGAGAL menginisialisasi satu atau lebih model penting.")

## 3. Unggah Gambar Sumber (Wajah Asli)

Unggah gambar yang berisi wajah yang ingin Anda **gunakan** sebagai wajah pengganti.

In [ ]:
source_image_cv = None 
source_face = None    # CRUCIAL: Variabel untuk menyimpan fitur wajah sumber

def upload_and_extract_source_face_img(face_analysis_app):
    print("Silakan unggah gambar untuk wajah sumber:")
    uploaded_files = files.upload()
    if not uploaded_files: return None, None

    img_name = list(uploaded_files.keys())[0]
    img_bytes = uploaded_files[img_name]
    nparr = np.frombuffer(img_bytes, np.uint8)
    img_cv = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img_cv is None: print(f"Gagal membaca '{img_name}'."); return None, None
    print(f"Gambar sumber '{img_name}' diunggah.")

    print("Mendeteksi wajah pada gambar sumber...")
    faces_in_source = face_analysis_app.get(img_cv)
    if not faces_in_source: print("ERROR: Tidak ada wajah terdeteksi di gambar sumber!"); return img_cv, None
    
    main_source_face = faces_in_source[0]
    print(f"Ditemukan {len(faces_in_source)} wajah. Menggunakan wajah pertama.")
    return img_cv, main_source_face

if 'app' in locals() and app is not None:
    source_image_cv, source_face = upload_and_extract_source_face_img(app)
    if source_face is not None:
        print("Fitur wajah sumber ('source_face') siap.")
        # Tampilkan gambar sumber (opsional)
        plt.imshow(cv2.cvtColor(source_image_cv, cv2.COLOR_BGR2RGB))
        plt.title('Gambar Sumber')
        plt.axis('off')
        plt.show()
    else:
        print("Gagal mendapatkan 'source_face'.")
else:
    print("Model 'app' belum siap.")

## 4. Unggah Gambar Target dan Lakukan Face Swap

Unggah gambar target (gambar di mana wajah akan diganti), lalu lakukan proses face swap.

In [ ]:
target_image_cv = None
result_image_cv = None

def upload_target_image_and_swap(source_face_features, face_analysis_app, face_swapping_model):
    if source_face_features is None: print("ERROR: 'source_face' belum ada."); return None
    if face_analysis_app is None or face_swapping_model is None: print("ERROR: Model 'app' atau 'swapper' belum siap."); return None

    print("Silakan unggah gambar target:")
    uploaded_target_files = files.upload()
    if not uploaded_target_files: print("Tidak ada gambar target diunggah."); return None

    target_img_name = list(uploaded_target_files.keys())[0]
    target_img_bytes = uploaded_target_files[target_img_name]
    nparr_target = np.frombuffer(target_img_bytes, np.uint8)
    trg_img_cv = cv2.imdecode(nparr_target, cv2.IMREAD_COLOR)

    if trg_img_cv is None: print(f"Gagal membaca gambar target '{target_img_name}'."); return None
    print(f"Gambar target '{target_img_name}' diunggah.")
    
    # Tampilkan gambar target (opsional)
    # plt.imshow(cv2.cvtColor(trg_img_cv, cv2.COLOR_BGR2RGB))
    # plt.title('Gambar Target Asli')
    # plt.axis('off')
    # plt.show()

    print("Mendeteksi wajah pada gambar target...")
    faces_in_target = face_analysis_app.get(trg_img_cv)
    if not faces_in_target: print("ERROR: Tidak ada wajah terdeteksi di gambar target!"); return trg_img_cv # Kembalikan asli jika tak ada wajah

    print(f"Ditemukan {len(faces_in_target)} wajah di gambar target. Memproses wajah pertama...")
    target_face_to_swap = faces_in_target[0] # Ambil wajah target pertama
    
    # CRUCIAL: Lakukan face swap
    result_img = trg_img_cv.copy() # Salin gambar target untuk dimodifikasi
    result_img = face_swapping_model.get(result_img, target_face_to_swap, source_face_features, paste_back=True)
    print("Face swap selesai.")
    return result_img

if 'source_face' in locals() and source_face is not None and \
   'app' in locals() and app is not None and \
   'swapper' in locals() and swapper is not None:
    
    result_image_cv = upload_target_image_and_swap(source_face, app, swapper)
    
    if result_image_cv is not None:
        print("\nMenampilkan Hasil Face Swap:")
        # Tampilkan gambar sumber, target (jika belum), dan hasil
        fig, ax = plt.subplots(1, 2, figsize=(12, 6)) # Atau 1,3 jika ingin tampilkan target asli juga
        if source_image_cv is not None:
            ax[0].imshow(cv2.cvtColor(source_image_cv, cv2.COLOR_BGR2RGB))
            ax[0].set_title('Wajah Sumber')
            ax[0].axis('off')
        else:
            ax[0].text(0.5, 0.5, 'Gambar Sumber Tidak Ada', ha='center', va='center')
            ax[0].axis('off')

        ax[1].imshow(cv2.cvtColor(result_image_cv, cv2.COLOR_BGR2RGB))
        ax[1].set_title('Hasil Face Swap')
        ax[1].axis('off')

        plt.tight_layout()
        plt.show()
        
        # Untuk menyimpan hasil:
        # output_filename_img = "hasil_faceswap_gambar.jpg"
        # cv2.imwrite(output_filename_img, result_image_cv)
        # print(f"Gambar hasil disimpan sebagai {output_filename_img}")
        # files.download(output_filename_img)
else:
    print("Pastikan 'source_face', 'app', dan 'swapper' sudah siap sebelum menjalankan sel ini.")

## 5. Penjelasan Tambahan

- **Kualitas Hasil**: Bergantung pada kualitas gambar input, kemiripan pose/pencahayaan, dan model.
- **Beberapa Wajah**: Kode ini menukar wajah pertama yang terdeteksi pada gambar target. Modifikasi diperlukan untuk menangani banyak wajah.
- **Penyimpanan**: Ada kode yang dikomentari untuk menyimpan dan mengunduh gambar hasil jika Anda mau.